In [45]:
pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.


In [46]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import google.generativeai as genai

In [47]:
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

if not api_key:
    print("Api key not found. Please set the GEMINI_API_KEY environment variable.")
else:
    print("API key loaded successfully.")

API key loaded successfully.


In [48]:
client = genai.GenerativeModel('gemini-2.5-pro')

In [49]:
# A class to represent a webpage

class Website:
    """
    A class to represent a website that we have scraped
    """
    def __init__(self, url):
        """
        Initializes the Website object with a URL and fetches its content.
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else 'No Title found'
        for irrelevant in soup(['script', 'style', 'img', 'input']):
            irrelevant.decompose()
        self.text = soup.get_text(separator='\n', strip=True)

In [50]:
page = Website("https://edwarddonner.com")
print(page.title)
print(page.text[:500])

Home - Edward Donner
Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
.


System Prompt - Tells what task they are performing and what tone they should use

A user prompt - The conversation starter that they should reply to

In [51]:
# Create a system prompt for the model

sys_prompt = "You are a helpful assistant that analyses and summarizes web pages \
        and provides a concise summary of the content.\
        Respond in markdown"

In [52]:
# A function that writes user prompt that asks for summarization

def user_prompt(website):
    prompt = f"""You are a helpful assistant that analyses and summarizes web pages.
    Please provide a concise summary of the content in markdown format.

    You are looking at a website titled: {website.title}.

    The content of this website is as follows:
    {website.text}
    """
    return prompt

In [55]:
def summarize_website(url):
    website = Website(url)

    # Create the prompt
    content = user_prompt(website)

    # Generate the content
    response = client.generate_content(content)

    # Return the text from the response
    return response.text

In [56]:
summarize_website("https://edwarddonner.com")

'This is the personal website of **Edward (Ed) Donner**, a technologist specializing in AI and Large Language Models (LLMs).\n\n*   **Professional Background**:\n    *   Co-founder and CTO of **Nebula.io**, an AI-powered platform used by recruiters to discover and manage talent.\n    *   Previously founded the AI startup **untapt**, which was acquired in 2021.\n\n*   **Interests and Expertise**:\n    *   He focuses on writing code and experimenting with LLMs.\n    *   His personal interests include DJing, electronic music production, and reading Hacker News.\n\n*   **Website Content**:\n    *   Features projects like **Outsmart**, a competitive arena for LLMs.\n    *   Includes a "Posts" section with articles and courses on topics like "Agentic AI Engineering" and "AI in Production."\n    *   Provides links to connect with him on social media and subscribe to his newsletter.'

In [57]:
# A Function to display this nicely in markdown

def display_summary(url):
    summary = summarize_website(url)
    display(Markdown(summary))

In [58]:
display_summary("https://edwarddonner.com")

This is the personal website of **Ed Donner**, a co-founder and CTO of an AI company called **Nebula.io**.

### Key Information:

*   **Professional Background:**
    *   Co-founder and CTO of **Nebula.io**, a company using proprietary AI and LLMs to help recruiters source and manage talent.
    *   Former founder and CEO of the AI startup **untapt**, which was acquired in 2021.
*   **Interests:**
    *   Writing code and experimenting with Large Language Models (LLMs).
    *   DJing and amateur electronic music production.
*   **Website Content:**
    *   **Projects:** Showcases "Connect Four" and "Outsmart," an arena where LLMs compete against each other.
    *   **Posts:** A blog section with articles on topics like Generative AI, Agentic AI engineering, and LLM courses.
    *   **Contact:** Provides links to his LinkedIn and Twitter profiles, his email address, and a newsletter subscription.